In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1) SDG Data

## Import data

In [2]:
df = pd.read_csv('world_bank_data.csv', na_values = '..')
df.head()

,Country Name,Country Code,Series Name,Series Code,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Singapore,SGP,"PM2.5 air pollution, mean annual exposure (mic...",EN.ATM.PM25.MC.M3,19.349421,20.886493,18.707071,17.408877,20.663389,18.797803,19.078256,NaN,NaN,NaN
1,Singapore,SGP,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,0.480000,0.510000,0.560000,0.590000,0.650000,0.680000,0.670000,0.730000,0.840000,NaN
2,Singapore,SGP,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,8.636708,8.224097,8.132744,8.117412,8.207766,8.221450,8.755842,8.406229,8.307079,NaN
3,Singapore,SGP,Energy intensity level of primary energy (MJ/$...,EG.EGY.PRIM.PP.KD,2.050000,2.310000,2.390000,2.520000,2.680000,2.660000,2.800000,2.510000,2.570000,NaN
4,Singapore,SGP,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,2.069550,1.918330,1.921040,2.082280,2.174450,2.074710,1.898720,1.813910,1.890640,NaN


## Clean data

In [4]:
# General descriptions -------
df.info()
df.describe()
print(df.shape)
df.columns

# NA values per year 
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   50 non-null     object 
 1   Country Code   48 non-null     object 
 2   Series Name    48 non-null     object 
 3   Series Code    48 non-null     object 
 4   2011 [YR2011]  42 non-null     float64
 5   2012 [YR2012]  41 non-null     float64
 6   2013 [YR2013]  42 non-null     float64
 7   2014 [YR2014]  41 non-null     float64
 8   2015 [YR2015]  42 non-null     float64
 9   2016 [YR2016]  41 non-null     float64
 10  2017 [YR2017]  42 non-null     float64
 11  2018 [YR2018]  34 non-null     float64
 12  2019 [YR2019]  34 non-null     float64
 13  2020 [YR2020]  9 non-null      float64
dtypes: float64(10), object(4)
memory usage: 5.9+ KB
(53, 14)


Country Name      3
Country Code      5
Series Name       5
Series Code       5
2011 [YR2011]    11
2012 [YR2012]    12
2013 [YR2013]    11
2014 [YR2014]    12
2015 [YR2015]    11
2016 [YR2016]    12
2017 [YR2017]    11
2018 [YR2018]    19
2019 [YR2019]    19
2020 [YR2020]    44
dtype: int64

In [15]:
# Clean data -------

# Remove irrelavant rows 
df_clean = df[~df['Country Code'].isna()]


# Remove rows that are entirely NA values 
id_cols = ['Country Name', 'Country Code', 'Series Name', 'Series Code']
year_cols = df_clean.columns.difference(id_cols)
df_clean = df_clean.dropna(subset=year_cols, how='all')
print(df_clean.shape) # check
print(df_clean.isna().sum()) # check remaining NA values


# Change column names
new_cols = list(df_clean.columns[:4]) + [str(y) for y in range(2011, 2021)]
df_clean.columns = new_cols
print(df_clean.columns) # check


# Change from wide to long format
df_long = df_clean.melt(
    id_vars = ['Country Name', 'Country Code', 'Series Name', 'Series Code'],
    value_vars = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
    var_name = 'year',
    value_name = 'value'
)

df_long['year'] = df_long['year'].astype(int) # change years to integers

print(df_long.head()) # check


# Simplify Series Name values 
df_long['Series Name'].unique()

mapping = {
    'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)' : 'PM2.5',
    'Renewable energy consumption (% of total final energy consumption)' : 'Renewable energy consumption',
    'CO2 emissions (metric tons per capita)' : 'CO2 emissions',
    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)' : 'Energy intensity',
    'Research and development expenditure (% of GDP)' : 'R&D',
    'Adjusted net savings, excluding particulate emission damage (% of GNI)' : 'Adjusted net savings'
}

df_long['Series Name'] = df_long['Series Name'].replace(mapping)
print(df_long['Series Name'].unique()) # check 


# Change country names
df_long['Country Name'].unique()

names_mapping = {
    'Hong Kong SAR, China' : 'Hong Kong',
     "Korea, Dem. People's Rep." : 'South Korea'
}

df_long['Country Name'] = df_long['Country Name'].replace(names_mapping)
print(df_long['Country Name'].unique()) # check


# Standardize column names

col_map = {
    'Country Name' : 'country_name',
    'Country Code' : 'country_code',
    'Series Name' : 'series_name',
    'Series Code' : 'series_code'
}

df_long = df_long.rename(columns=col_map)
print(df_long.columns) # check


# Remaining NA values
df_long.isna().sum() # we will retain these NA values - most come from the year 2020 where data is missing due to COVID 19
df_long["value"] = df_long["value"].where(df_long["value"].notna(), None) # changing NA values to None so that they can be converted to NULL when imported into SQL

(42, 14)
Country Name      0
Country Code      0
Series Name       0
Series Code       0
2011 [YR2011]     0
2012 [YR2012]     1
2013 [YR2013]     0
2014 [YR2014]     1
2015 [YR2015]     0
2016 [YR2016]     1
2017 [YR2017]     0
2018 [YR2018]     8
2019 [YR2019]     8
2020 [YR2020]    33
dtype: int64
Index(['Country Name', 'Country Code', 'Series Name', 'Series Code', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')
  Country Name Country Code  \
0    Singapore          SGP   
1    Singapore          SGP   
2    Singapore          SGP   
3    Singapore          SGP   
4    Singapore          SGP   

                                         Series Name        Series Code  year  \
0  PM2.5 air pollution, mean annual exposure (mic...  EN.ATM.PM25.MC.M3  2011   
1  Renewable energy consumption (% of total final...     EG.FEC.RNEW.ZS  2011   
2             CO2 emissions (metric tons per capita)     EN.ATM.CO2E.PC  2011   
3  Energ

## Export data to csv for SQL analysis

In [16]:
df_long.to_csv("sdg_long_clean.csv", index=False, na_rep='NULL')

# 2) GDP Data

## Import GDP Data

In [5]:
df2 = pd.read_csv('gdp_wb_data.csv')
df2.head()

,Country Name,Country Code,Series Name,Series Code,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Singapore,SGP,GDP per capita (current US$),NY.GDP.PCAP.CD,53890.4287270504,55546.4885386921,56967.4257940383,57562.5307937678,55646.6187469505,56860.4132375207,61150.7271966595,66859.3383447804,65831.1894308765,60729.4503486794
1,Singapore,SGP,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,4.02338164865897,1.90446223662212,3.13397914899265,2.59472315486744,1.76232468638902,2.22673004020686,4.56875082144181,3.17554509933274,-0.0545548569257619,-3.8436564563571
2,United States,USA,GDP per capita (current US$),NY.GDP.PCAP.CD,50065.9665041742,51784.4185738837,53291.1276891406,55123.8497869046,56762.7294515989,57866.7449341091,59914.777796976,62805.2537579917,65094.7994287929,63027.6795267153
3,United States,USA,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,0.814519357932042,1.53310203539129,1.13869234666606,1.54038064866397,1.95300411790625,0.933375361665711,1.61080808257104,2.37847922290794,1.82412448165672,-4.32861775376968
4,United Kingdom,GBR,GDP per capita (current US$),NY.GDP.PCAP.CD,42284.8844902996,42686.8000524926,43713.8141242308,47787.2412984884,45404.5677734722,41499.5557033073,40857.7555829627,43646.9519711493,43070.4983595888,41098.0786527828


## Clean data

In [10]:
# Clean data -------

# Remove irrelavant rows 
df2 = df2[~df2['Country Code'].isna()]

# Change column names
new_cols = list(df2.columns[:4]) + [str(y) for y in range(2011, 2021)]
df2.columns = new_cols
print(df2.columns) # check


# Change from wide to long format
df2_long = df2.melt(
    id_vars = ['Country Name', 'Country Code', 'Series Name', 'Series Code'],
    value_vars = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
    var_name = 'year',
    value_name = 'value'
)

df2_long['year'] = df2_long['year'].astype(int) # change years to integers

print(df2_long.head()) # check


# Simplify Series Name values 
df2_long['Series Name'].unique()

mapping = {
    'GDP per capita (current US$)' : 'GDP per capita',
    'GDP per capita growth (annual %)' : 'GDP per capita growth'
}

df2_long['Series Name'] = df2_long['Series Name'].replace(mapping)
print(df2_long['Series Name'].unique()) # check 


# Change country names

names_mapping = {
    'Hong Kong SAR, China' : 'Hong Kong',
     "Korea, Dem. People's Rep." : 'South Korea'
}

df2_long['Country Name'] = df2_long['Country Name'].replace(names_mapping)
print(df2_long['Country Name'].unique()) # check


# Standardize column names

col_map = {
    'Country Name' : 'country_name',
    'Country Code' : 'country_code',
    'Series Name' : 'series_name',
    'Series Code' : 'series_code'
}

df2_long = df2_long.rename(columns=col_map)
print(df2_long.columns) # check

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')
     Country Name Country Code                       Series Name  \
0       Singapore          SGP      GDP per capita (current US$)   
1       Singapore          SGP  GDP per capita growth (annual %)   
2   United States          USA      GDP per capita (current US$)   
3   United States          USA  GDP per capita growth (annual %)   
4  United Kingdom          GBR      GDP per capita (current US$)   

         Series Code  year              value  
0     NY.GDP.PCAP.CD  2011   53890.4287270504  
1  NY.GDP.PCAP.KD.ZG  2011   4.02338164865897  
2     NY.GDP.PCAP.CD  2011   50065.9665041742  
3  NY.GDP.PCAP.KD.ZG  2011  0.814519357932042  
4     NY.GDP.PCAP.CD  2011   42284.8844902996  
['GDP per capita' 'GDP per capita growth']
['Singapore' 'United States' 'United Kingdom' 'Japan' 'South Korea'
 'Australia' 

## Export data to csv for analysis

In [11]:
df2_long.to_csv("gdp_long_clean.csv", index=False, na_rep='NULL')